In [7]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss

from sklearn.ensemble import RandomForestClassifier

from urllib.request import Request, urlopen  # Python 3
import json
import os.path

import math
from datetime import timedelta
from datetime import datetime

In [8]:
API_Key = open("../variables/google_directions_query.txt", 'r').read()

In [9]:
data_dir = '../data/'
df_Merged_agg = pd.read_csv(data_dir + 'blockgroup_pairs_kc.csv')
print (df_Merged_agg)

                         bg_pair          o_bg          d_bg   lat_dest  \
0      530330001001-530330001005  530330001001  530330001005  47.722930   
1      530330001001-530330007001  530330001001  530330007001  47.717444   
2      530330001001-530330042003  530330001001  530330042003  47.664708   
3      530330001001-530330053023  530330001001  530330053023  47.655764   
4      530330001001-530330067001  530330001001  530330067001  47.632382   
5      530330001002-530330001003  530330001002  530330001003  47.722811   
6      530330001002-530330007001  530330001002  530330007001  47.717444   
7      530330001002-530330018001  530330001002  530330018001  47.694197   
8      530330001002-530330018002  530330001002  530330018002  47.692224   
9      530330001002-530330093002  530330001002  530330093002  47.589459   
10     530330001002-530330203005  530330001002  530330203005  47.772873   
11     530330001002-530330323193  530330001002  530330323193  47.759837   
12     530330001003-53033

In [10]:
def downloadPolylines():

    # Load origin / destination table
    data_dir = '../data/'
    df_Merged_agg = pd.read_csv(data_dir + 'blockgroup_pairs_kc.csv')
    #df_Merged_agg = df_Merged_agg.iloc[0:100]
    df_Merged_agg = df_Merged_agg[df_Merged_agg['o_bg'] != df_Merged_agg['d_bg']]

    # Access API and save polylines
    df_Polylines = pd.DataFrame()
    Bg_pair = []
    O_bg = []
    D_bg = []
    Polyline = []

    for index, row in df_Merged_agg.iterrows():
        # Build the Origin and Destination strings
        Origin = str(row["lat_origin"]) + "," + str(row["lon_origin"])
        Destination = str(row["lat_dest"]) + "," + str(row["lon_dest"])
        URL = "https://maps.googleapis.com/maps/api/directions/json?&origin=" + Origin + \
                        "&destination=" + Destination + "&key=" + API_Key
        print (index)
        q = Request(URL)
        a = urlopen(q).read()
        data = json.loads(a)
        sortOrder = 0  
        
        if 'errorZ' in data:
            print (data["error"])
        
        Polyline.append(data['routes'][0]['overview_polyline']['points'])
        Bg_pair.append(row["bg_pair"])
        O_bg.append(row["o_bg"])
        D_bg.append(row["d_bg"])

    df_Polylines['bg_pair'] = Bg_pair
    df_Polylines['polyline'] = Polyline
    df_Polylines['o_bg'] = Polyline
    df_Polylines['d_bg'] = Polyline

    df_Polylines.to_csv(data_dir + 'BG_pair_polylines_kc.csv', mode='w', header=True, index=False)

#downloadPolylines()


In [11]:
## BUILD SEGMENT TABLE 

# Load origin / destination table
data_dir = '../data/'
df_Merged_agg = pd.read_csv(data_dir + 'BG_pair_polylines_kc.csv')
#df_Merged_agg = df_Merged_agg.iloc[0:100]

O_BG = []
D_BG = []

for index, row in df_Merged_agg.iterrows():   

    O_BG.append(row["bg_pair"].split("-")[0])
    D_BG.append(row["bg_pair"].split("-")[1])

df_Merged_agg['o_bg'] = O_BG
df_Merged_agg['d_bg'] = D_BG

print (df_Merged_agg)

#df_Merged_agg = df_Merged_agg[df_Merged_agg['o_bg'] != df_Merged_agg['d_bg']]

# Load polyline table, merge with o/d table
#df_BG_pair_polylines = pd.read_csv(data_dir + 'Trip_Household_Merged.csv')
#df_Merged_agg= pd.merge(left=df_Merged_agg, right=df_BG_pair_polylines, how='left', left_on='bg_pair', right_on='bg_pair')
df_Merged_agg = df_Merged_agg[df_Merged_agg['o_bg'] != df_Merged_agg['d_bg']]

df_Polylines = pd.DataFrame()
    
Latitude = []
Longitude = []
SortOrder = []
End = []
Bg_pair = []
Bg_origin = []
Bg_dest = []
SegmentID = []
SegmentDist = []
SegmentPct = []

tripID = 0
sortOrder = 0

for index, row in df_Merged_agg.iterrows():
    # Build the Origin and Destination strings
    
    polyline = []
    polyline =  decode_polyline(row["polyline"])
    segmentID = 0
    cumulativeDist = 0
    startLat = 0
    endLat = 0
    startLong = 0
    endLong = 0
    
    for i, coordinate in enumerate(polyline):
        
        # first point
        if i == 0:
            startLat = coordinate[0]
            startLong = coordinate[1]
            
            Latitude.append(startLat)
            Longitude.append(startLong)
            SortOrder.append(0)
            Bg_pair.append(row['bg_pair'])
            Bg_origin.append(row['o_bg'])
            Bg_dest.append(row['d_bg'])
            SegmentID.append(segmentID)
            cumulativeDist += ((math.sqrt((coordinate[0] - startLat)**2 + (coordinate[1] - startLong)**2))*69)
            SegmentDist.append(cumulativeDist)

        # not last point
        if i > 0 and i < len(polyline):

            # end of point
            Latitude.append(coordinate[0])
            Longitude.append(coordinate[1])
            SortOrder.append(1)
            Bg_pair.append(row['bg_pair'])
            Bg_origin.append(row['o_bg'])
            Bg_dest.append(row['d_bg'])
            SegmentID.append(segmentID)
            cumulativeDist += ((math.sqrt((coordinate[0] - startLat)**2 + (coordinate[1] - startLong)**2))*69)
            SegmentDist.append(cumulativeDist)            
    
            segmentID = segmentID + 1     
            
            # start of point         
            Latitude.append(coordinate[0])
            Longitude.append(coordinate[1])
            SortOrder.append(0)
            Bg_pair.append(row['bg_pair'])
            Bg_origin.append(row['o_bg'])
            Bg_dest.append(row['d_bg'])
            SegmentID.append(segmentID)
            startLat = coordinate[0]
            startLong = coordinate[1]
            cumulativeDist += ((math.sqrt((coordinate[0] - startLat)**2 + (coordinate[1] - startLong)**2))*69)
            SegmentDist.append(cumulativeDist)
            
        # last point
        if i == len(polyline):
        
            Latitude.append(coordinate[0])
            Longitude.append(coordinate[1])
            SortOrder.append(1)
            Bg_pair.append(row['bg_pair'])
            Bg_origin.append(row['o_bg'])
            Bg_dest.append(row['d_bg'])
            SegmentID.append(segmentID)
            cumulativeDist += ((math.sqrt((coordinate[0] - startLat)**2 + (coordinate[1] - startLong)**2))*69)
            SegmentDist.append(cumulativeDist)
            
    segmentID = segmentID + 1   
                    
df_Polylines['lat'] = Latitude
df_Polylines['lon'] = Longitude
df_Polylines['sort_order'] = SortOrder
df_Polylines['bg_pair'] = Bg_pair
df_Polylines['o_bg'] = Bg_origin
df_Polylines['d_bg'] = Bg_dest
df_Polylines['segmentDist'] = SegmentDist
df_Polylines['segmentID'] = SegmentID 

#print (df_Polylines)
df_Polylines.to_csv(data_dir + 'Living_Graph2.csv', mode='w', header=True, index=False)

# Add segment percentages

df_Polylines_Agg = df_Polylines.groupby(['bg_pair'], as_index=False).agg({'segmentDist':'max'})

df_Polylines= pd.merge(left=df_Polylines, right=df_Polylines_Agg, how='left', left_on='bg_pair', right_on='bg_pair')
df_Polylines['segmentPct'] = df_Polylines['segmentDist_x'] / df_Polylines['segmentDist_y']

print (df_Polylines)

                         bg_pair  \
0      530330001001-530330001005   
1      530330001001-530330007001   
2      530330001001-530330042003   
3      530330001001-530330053023   
4      530330001001-530330067001   
5      530330001002-530330001003   
6      530330001002-530330007001   
7      530330001002-530330018001   
8      530330001002-530330018002   
9      530330001002-530330093002   
10     530330001002-530330203005   
11     530330001002-530330323193   
12     530330001003-530330001002   
13     530330001003-530330001005   
14     530330001003-530330002002   
15     530330001003-530330002004   
16     530330001003-530330002005   
17     530330001003-530330007001   
18     530330001003-530330007002   
19     530330001003-530330009001   
20     530330001003-530330012005   
21     530330001003-530330026003   
22     530330001003-530330043021   
23     530330001003-530330052005   
24     530330001003-530330071002   
25     530330001003-530330079002   
26     530330001003-53033008

              lat        lon  sort_order                    bg_pair  \
0        47.72640 -122.28470           0  530330001001-530330001005   
1        47.72643 -122.28685           1  530330001001-530330001005   
2        47.72643 -122.28685           0  530330001001-530330001005   
3        47.72644 -122.28884           1  530330001001-530330001005   
4        47.72644 -122.28884           0  530330001001-530330001005   
5        47.72647 -122.29092           1  530330001001-530330001005   
6        47.72647 -122.29092           0  530330001001-530330001005   
7        47.72647 -122.29224           1  530330001001-530330001005   
8        47.72647 -122.29224           0  530330001001-530330001005   
9        47.72629 -122.29226           1  530330001001-530330001005   
10       47.72629 -122.29226           0  530330001001-530330001005   
11       47.72554 -122.29236           1  530330001001-530330001005   
12       47.72554 -122.29236           0  530330001001-530330001005   
13    

In [12]:
# Merge with Trip data
df_trip_merged = pd.read_csv(data_dir + 'Trip_Household_Merged.csv')
df_trips = df_trip_merged[['o_bg','d_bg','depart_time_hhmm','google_duration','gender','mode','depart_time','tripid','trip_weight_revised']]
df_trips['bg_pair'] = df_trips['o_bg'].astype(str) + "-" + df_trips['d_bg'].astype(str)

# merge polyline data with trip data
df_Polylines_trips = pd.merge(left=df_Polylines, right=df_trips, how='left', left_on='bg_pair', right_on='bg_pair')

# calculate hhmm by adding duration * segmentPct
#df_Polylines['legMinutes'] = df_Polylines['google_duration'] * df_Polylines['segmentPct2']
df_Polylines_trips['depart_time_hhmm'] = pd.to_datetime(df_Polylines_trips['depart_time_hhmm'])

# scale google duration by mode type
#df_Polylines_trips['google_duration'] = np.where(df_Polylines_trips['mode']=='transit', 
#                df_Polylines_trips['google_duration']*1.5, df_Polylines_trips['google_duration'])

#df_Polylines_trips['google_duration'] = np.where(df_Polylines_trips['mode']=='walk', 
#                df_Polylines_trips['google_duration']*7, df_Polylines_trips['google_duration'])

#df_Polylines_trips['google_duration'] = np.where(df_Polylines_trips['mode']=='bike', 
#                df_Polylines_trips['google_duration']*2.5, df_Polylines_trips['google_duration'])

# calculate increments in one-minute intervals.
df_Polylines_trips['increment'] = df_Polylines_trips['segmentPct'] * df_Polylines_trips['google_duration']
df_Polylines_trips['increment'] = df_Polylines_trips['increment'].astype(int)

#df_Polylines_trips['timestamp'] = df_Polylines_trips['depart_time_hhmm'] + timedelta(minutes=x)

def f(x):    
    return x['depart_time_hhmm'] + timedelta(minutes=x['increment'])

df_Polylines_trips['timestamp'] = df_Polylines_trips.apply(f, axis=1)

# Save file for segment visualization
df_Polylines_trips.to_csv(data_dir + 'Living_Graph_Trips_Segment.csv', mode='w', header=True, index=False)

# filter for point visualization
df_Polylines_trips = df_Polylines_trips[(df_Polylines_trips["sort_order"] == 1) | (df_Polylines_trips["segmentID"] == 0)] 

# Identify the first segment in each minute increment
First_minute = []
trip_pair = ""
previous_trip_pair = ""

for index, row in df_Polylines_trips.iterrows():  

    trip_pair = str(row['tripid']) + str(row['increment'])
    if trip_pair == previous_trip_pair:
        First_minute.append(0)
    else:
        First_minute.append(1)
    previous_trip_pair = trip_pair
df_Polylines_trips['first_minute'] = First_minute
print (df_Polylines_trips)
# Save file for visualization
df_Polylines_trips.to_csv(data_dir + 'Living_Graph_Trips_Point.csv', mode='w', header=True, index=False)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


              lat        lon  sort_order                    bg_pair  \
0        47.72640 -122.28470           0  530330001001-530330001005   
1        47.72643 -122.28685           1  530330001001-530330001005   
3        47.72644 -122.28884           1  530330001001-530330001005   
5        47.72647 -122.29092           1  530330001001-530330001005   
7        47.72647 -122.29224           1  530330001001-530330001005   
9        47.72629 -122.29226           1  530330001001-530330001005   
11       47.72554 -122.29236           1  530330001001-530330001005   
13       47.72508 -122.29244           1  530330001001-530330001005   
15       47.72468 -122.29254           1  530330001001-530330001005   
17       47.72445 -122.29261           1  530330001001-530330001005   
19       47.72439 -122.29266           1  530330001001-530330001005   
21       47.72437 -122.29267           1  530330001001-530330001005   
23       47.72362 -122.29294           1  530330001001-530330001005   
25    

In [13]:
# Decode polyline
def decode_polyline(polyline_str):
    '''Pass a Google Maps encoded polyline string; returns list of lat/lon pairs'''
    index, lat, lng = 0, 0, 0
    coordinates = []
    changes = {'latitude': 0, 'longitude': 0}

    # Coordinates have variable length when encoded, so just keep
    # track of whether we've hit the end of the string. In each
    # while loop iteration, a single coordinate is decoded.
    while index < len(polyline_str):
        # Gather lat/lon changes, store them in a dictionary to apply them later
        for unit in ['latitude', 'longitude']: 
            shift, result = 0, 0

            while True:
                byte = ord(polyline_str[index]) - 63
                index+=1
                result |= (byte & 0x1f) << shift
                shift += 5
                if not byte >= 0x20:
                    break

            if (result & 1):
                changes[unit] = ~(result >> 1)
            else:
                changes[unit] = (result >> 1)

        lat += changes['latitude']
        lng += changes['longitude']

        coordinates.append((lat / 100000.0, lng / 100000.0))

    return coordinates
